imports

In [2]:
import torch
import os
import pandas as pd
import numpy as np
import plotnine

from HGNN.train.configParser import ConfigParser
from HGNN.train import CNN, dataLoader

experimentsFileName = "experiments.csv"

parameters

In [3]:
experimentsPath="/home/jnl47/Documents/dev/HGNN/experiments/"
dataPath="/home/jnl47/Documents/dev/HGNN/data"
experimentName="allgraded"
trial_hash="cf5bb78c6fcc769776b2af4bf2d86d7be5835a7f9c4face14827d44f"

cuda=1

cuda

In [4]:
# set cuda
if torch.cuda.is_available():
    torch.cuda.set_device(cuda)
    print("using cuda", cuda)

using cuda 1


Get dataset

In [5]:
experimentsFileNameAndPath = os.path.join(experimentsPath, experimentsFileName)
if os.path.exists(experimentsFileNameAndPath):
    experiments_df = pd.read_csv(experimentsFileNameAndPath)
experiments_df["trialHash"]
experiments_df[experiments_df["trialHash"] == trial_hash]

,experimentName,modelName,datasetName,experimentHash,trialHash,image_path,suffix,training_count,validation_count,batchSize,...,learning_rate,numOfTrials,patience,fc_width,fc_layers,modelType,lambda,unsupervisedOnTest,tl_model,augmented
60,allgraded,models/cf5bb78c6fcc769776b2af4bf2d86d7be5835a7...,datasplits/4fc30fc78733889043b62f0c6b4fa76b373...,b6fa90f577bbf358bafbc9be0871fad9ffbc5b5d730248...,cf5bb78c6fcc769776b2af4bf2d86d7be5835a7f9c4fac...,INHS_segmented_padded_fish,allgraded,0.64,0.16,32,...,0.0001,5,50,200,1,BB,0.6,False,ResNet18,False


In [6]:
# Get experiment parameters
experimentsFileNameAndPath = os.path.join(experimentsPath, experimentsFileName)
if os.path.exists(experimentsFileNameAndPath):
    experiments_df = pd.read_csv(experimentsFileNameAndPath)
else:
    raise Exception("Experiment not " + trial_hash + " found!")
experimentRecord = experiments_df[experiments_df["trialHash"] == trial_hash]
experiment_params = experimentRecord.to_dict('records')[0]
print(experiment_params)

config_parser = ConfigParser(experimentsPath, dataPath, experimentName)
experimentPathAndName = os.path.join(experimentsPath, experimentName)
datasetManager = dataLoader.datasetManager(experimentPathAndName)
datasetManager.updateParams(config_parser.fixPaths(experiment_params))
dataset = datasetManager.getDataset()
train_loader, validation_loader, test_loader = datasetManager.getLoaders()

{'experimentName': 'allgraded', 'modelName': 'models/cf5bb78c6fcc769776b2af4bf2d86d7be5835a7f9c4face14827d44f', 'datasetName': 'datasplits/4fc30fc78733889043b62f0c6b4fa76b3733a07b1079ee11fb2eb148', 'experimentHash': 'b6fa90f577bbf358bafbc9be0871fad9ffbc5b5d730248e89ce7f4f2', 'trialHash': 'cf5bb78c6fcc769776b2af4bf2d86d7be5835a7f9c4face14827d44f', 'image_path': 'INHS_segmented_padded_fish', 'suffix': 'allgraded', 'training_count': 0.64, 'validation_count': 0.16, 'batchSize': 32, 'n_epochs': 5000, 'learning_rate': 0.0001, 'numOfTrials': 5, 'patience': 50, 'fc_width': 200, 'fc_layers': 1, 'modelType': 'BB', 'lambda': 0.6, 'unsupervisedOnTest': False, 'tl_model': 'ResNet18', 'augmented': False}
Creating dataset...


Loading images: 100%|██████████| 1703/1703 [02:17<00:00, 12.35it/s, fileName=INHS_FISH_97787.jpg] 

Creating dataset... Done.
Loading saved indices...
file /home/jnl47/Documents/dev/HGNN/experiments/allgraded/datasplits/b270091ac751885bcba217b9acffac4fdd2fe4f62950f52fd8930511/trainingIndex.csv read
file /home/jnl47/Documents/dev/HGNN/experiments/allgraded/datasplits/b270091ac751885bcba217b9acffac4fdd2fe4f62950f52fd8930511/valIndex.csv read
file /home/jnl47/Documents/dev/HGNN/experiments/allgraded/datasplits/b270091ac751885bcba217b9acffac4fdd2fe4f62950f52fd8930511/testIndex.csv read
Creating loaders...
Creating loaders... Done.


Get untrained model

In [7]:
architecture = {
    "fine": len(dataset.csv_processor.getFineList()),
    "coarse" : len(dataset.csv_processor.getCoarseList())
}
dataset.csv_processor.getFineList()
model = CNN.create_model(architecture, experiment_params)
model
# get the model and the parameters
modelName = experimentRecord.iloc[0]["modelName"]
trialName = os.path.join(experimentPathAndName, modelName)
_ = CNN.loadModel(model, trialName)

In [8]:
dataset.csv_processor.getCoarseList()

['Notropis',
 'Esox',
 'Gambusia',
 'Lepomis',
 'Noturus',
 'Phenacobius',
 'Cyprinus',
 'Carassius']

In [9]:
model

CNN_hierarchy(
  (pretrained_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

In [10]:
modelName

'models/cf5bb78c6fcc769776b2af4bf2d86d7be5835a7f9c4face14827d44f'

sort through predictions

In [11]:
df_misclassified = pd.DataFrame(columns=['file name', 'true label', 'probability of true label', 'predicted label'])
df_correctlyclassified = pd.DataFrame(columns=['file name', 'true label', 'probability of true label', 'predicted label'])

# get probability of correct prediction and true label
predProblist, lbllist = CNN.getLoaderPredictionProbabilities(test_loader, model, experiment_params)
_, predlist = torch.max(predProblist, 1)
lbllist = lbllist.reshape(lbllist.shape[0], -1)
predProblist = predProblist.gather(1, lbllist)
predProblist = predProblist.reshape(1, -1)
predProblist = predProblist[0]

# sort through
predProblist, indices = torch.sort(predProblist)
predlist = predlist[indices]
lbllist = lbllist[indices]

for i, lbl in enumerate(lbllist):
    prd = predlist[i]
    prdProb = predProblist[i]
    
    if torch.cuda.is_available():
        lbl = lbl.cpu()
        prd = prd.cpu()
        prdProb = prdProb.cpu()

    s = dataset[i]
    row = {'file name' : s['fileName'] , 
           'true label' : int(lbl.numpy()), 
           'probability of true label': float(prdProb.numpy()),
           'predicted label' : int(prd.numpy())}
    
    if(lbl != prd):
        df_misclassified = df_misclassified.append(row, ignore_index=True)
    else:
        df_correctlyclassified = df_correctlyclassified.append(row, ignore_index=True)
        
df_misclassified = df_misclassified.sort_values(by=[ 'true label', 'probability of true label'])
df_correctlyclassified = df_correctlyclassified.sort_values(by=['true label', 'probability of true label'])

Transforming images: 100%|██████████| 1703/1703 [00:21<00:00, 78.66it/s]


In [12]:
len(df_misclassified)

63

In [13]:
len(df_correctlyclassified)

278

Display and save mispredicted

In [14]:
print(df_misclassified)
df_misclassified.to_csv(os.path.join(experimentPathAndName, modelName, 'misclassified examples.csv'))

               file name true label  probability of true label predicted label
41   INHS_FISH_33867.jpg          0                   0.133759              12
45  INHS_FISH_108440.jpg          0                   0.172629              14
50    INHS_FISH_9769.jpg          0                   0.213458               2
23   INHS_FISH_97116.jpg          1                   0.040090               5
2    INHS_FISH_28594.jpg          3                   0.001444              12
..                   ...        ...                        ...             ...
52   INHS_FISH_99134.jpg         13                   0.242304               8
3    INHS_FISH_99505.jpg         14                   0.002454               0
4    INHS_FISH_20964.jpg         14                   0.002781               0
33   INHS_FISH_98989.jpg         14                   0.080536              12
35   INHS_FISH_98093.jpg         14                   0.087230               0

[63 rows x 4 columns]


Save and display correctly predicted

Filename of correctly classified

In [15]:
print(df_correctlyclassified)
df_correctlyclassified.to_csv(os.path.join(experimentPathAndName, modelName, 'correctly classified examples.csv'))

               file name true label  probability of true label predicted label
36   INHS_FISH_99119.jpg          0                   0.535152               0
60   INHS_FISH_12001.jpg          0                   0.700548               0
74   INHS_FISH_99816.jpg          0                   0.780972               0
86    INHS_FISH_9871.jpg          0                   0.835673               0
100  INHS_FISH_64283.jpg          0                   0.876908               0
..                   ...        ...                        ...             ...
96   INHS_FISH_64639.jpg         15                   0.863661              15
17   INHS_FISH_97576.jpg         16                   0.393870              16
31   INHS_FISH_97062.jpg         16                   0.477894              16
34   INHS_FISH_62299.jpg         16                   0.522556              16
43   INHS_FISH_99274.jpg         16                   0.593848              16

[278 rows x 4 columns]


In [16]:
#left join against scores
meta=pd.read_csv("../../data/INHS_corrected_2020.06.09.csv")

#annotated webform data
qual=pd.read_csv("../../data/hdrwebform_20200728.csv") 

In [17]:
qual=qual.drop(columns=["fixed_scientific_name","scientific_name","resolution"]) #all NAN

In [18]:
moige=qual.set_index('image_name').join(meta.set_index('fileName')).join(df_correctlyclassified.set_index('file name'))

In [19]:
df_correctlyclassified.set_index('file name').join(qual.set_index('image_name'),how='inner')
df_misclassified.set_index('file name').join(qual.set_index('image_name'),how='inner')
df_correctlyclassified['classified']='correct'
df_misclassified['classified']='misclassified'
df_bothclassified=df_correctlyclassified.append(df_misclassified)

In [71]:
df_bothclassified
moige=df_bothclassified.set_index('file name').join(qual.set_index('image_name')).join(meta.set_index('fileName'))

In [126]:
moige['scientificName'].unique()

array(['Noturus flavus', 'Notropis atherinoides', 'Lepomis cyanellus',
       'Phenacobius mirabilis', 'Lepomis humilis', 'Esox americanus',
       'Gambusia affinis', 'Lepomis macrochirus', 'Lepomis megalotis',
       'Lepomis gibbosus', 'Notropis stramineus', 'Notropis buccata',
       'Notropis dorsalis', 'Notropis blennius', 'Cyprinus carpio',
       'Notropis hudsonius', 'Noturus gyrinus'], dtype=object)

In [108]:
from plotnine import *
moige=moige.rename(columns={'if_bent':'if_curved'})
res=moige.groupby('classified').mean().drop(['height','width','fish_number','if_fish','fixedCatalogNumber','checked_flag','landmark_failures_number','if_label_catalog_number_correct','if_label_name_correct','if_each_fish_label'],axis=1)

for incol in [col for col in moige if col.startswith('if')]:
    moige[incol] = moige[incol].astype(float)
flip_ifs = ['if_blur']
for col in flip_ifs:
    res[col] = 1-res[col]
res.transpose()
#ggplot(moige,aes('classified','if_fish'))+geom_bar()

classified,correct,misclassified
probability of true label,0.837085,0.105787
if_ruler,1.000000,1.000000
if_colorbar,0.003597,0.000000
if_label,1.000000,1.000000
if_overlapping,0.104317,0.126984
if_curved,0.017986,0.063492
if_missing_parts,0.010791,0.015873
if_blur,0.000000,0.000000
if_color_issue,0.000000,0.000000
specimen_angled,8.830935,8.841270


In [41]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [116]:


X=moige[['if_curved','if_missing_parts','image_quality','if_overlapping','if_background_uniform','if_parts_visible']]
y=(moige['classified']=='correct').astype(int)


In [117]:
X[['if_curved','if_missing_parts','if_background_uniform','if_parts_visible']].astype(int)


,if_curved,if_missing_parts,if_background_uniform,if_parts_visible
INHS_FISH_101846.jpg,0,0,0,1
INHS_FISH_102196.jpg,0,0,1,1
INHS_FISH_106829.jpg,0,0,0,1
INHS_FISH_107545.jpg,0,0,0,1
INHS_FISH_108440.jpg,0,0,0,1
...,...,...,...,...
INHS_FISH_99862.jpg,0,0,1,1
INHS_FISH_99867.jpg,0,0,1,1
INHS_FISH_99878.jpg,0,0,1,0
INHS_FISH_99922.jpg,0,0,1,0


In [118]:
y

INHS_FISH_101846.jpg    1
INHS_FISH_102196.jpg    1
INHS_FISH_106829.jpg    1
INHS_FISH_107545.jpg    1
INHS_FISH_108440.jpg    0
                       ..
INHS_FISH_99862.jpg     1
INHS_FISH_99867.jpg     1
INHS_FISH_99878.jpg     1
INHS_FISH_99922.jpg     1
INHS_FISH_99935.jpg     1
Name: classified, Length: 341, dtype: int64

In [119]:
import statsmodels.api as sm
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
X_imputed=imp.fit_transform(X)
X_imputed_df = pd.DataFrame(X_imputed, columns = X.columns, index = X.index)

logit_model=sm.Logit(y,X_imputed_df)
result=logit_model.fit()
print(result.summary2())


Optimization terminated successfully.
         Current function value: 0.472295
         Iterations 6
                           Results: Logit
Model:                 Logit             Pseudo R-squared:  0.013   
Dependent Variable:    classified        AIC:               334.1051
Date:                  2020-08-02 20:48  BIC:               357.0964
No. Observations:      341               Log-Likelihood:    -161.05 
Df Model:              5                 LL-Null:           -163.18 
Df Residuals:          335               LLR p-value:       0.51451 
Converged:             1.0000            Scale:             1.0000  
No. Iterations:        6.0000                                       
--------------------------------------------------------------------
                       Coef.  Std.Err.    z    P>|z|   [0.025 0.975]
--------------------------------------------------------------------
if_curved             -1.1098   0.7075 -1.5685 0.1168 -2.4965 0.2770
if_missing_parts       0.166

Leave out image_quality as that is redundant

In [120]:
X_no_iq=X_imputed_df.drop('image_quality',axis=1)
logit_model=sm.Logit(y,X_no_iq)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.496584
         Iterations 6
                           Results: Logit
Model:                 Logit             Pseudo R-squared:  -0.038  
Dependent Variable:    classified        AIC:               348.6704
Date:                  2020-08-02 20:48  BIC:               367.8298
No. Observations:      341               Log-Likelihood:    -169.34 
Df Model:              4                 LL-Null:           -163.18 
Df Residuals:          336               LLR p-value:       1.0000  
Converged:             1.0000            Scale:             1.0000  
No. Iterations:        6.0000                                       
--------------------------------------------------------------------
                       Coef.  Std.Err.    z    P>|z|   [0.025 0.975]
--------------------------------------------------------------------
if_curved             -0.4206   0.7073 -0.5947 0.5521 -1.8068 0.9656
if_missing_parts      -0.021